# Dependencies

In [148]:
!pip install -qU langchain-groq
!pip install -U ddgs
!pip install -qU duckduckgo-search langchain-community
!pip install -qU langchain-community wikipedia
!pip install -qU youtube_search
!pip install -qU  youtube-transcript-api
!pip install pytube

# LLM

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key="my_api_key"
)

# Tools

In [158]:
tools = []

Searching on web using duck duck go search.


In [159]:
from langchain_community.tools import DuckDuckGoSearchRun

tools.append(DuckDuckGoSearchRun())

Retreving content from wikipedia using WikipediaQueryRun

In [160]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikiSearch = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools.append(wikiSearch)

YouTube search tool using YoutubeLoader and YouTubeSearchTool

In [171]:
import ast # convert string into actual python list
from langchain_core.tools import tool
from langchain_community.tools import YouTubeSearchTool
from langchain_community.document_loaders import YoutubeLoader

@tool(
    "youtube_search",
    description=(
        "Search YouTube and return SHORT summaries of a few videos "
        "for the given query (not full transcripts because the model used has limited access tokens)."
    ),
)
def fetch_youtube_content(query: str) -> str:
    """
    Returns limited text from a few YouTube videos, to stay within token limits.
    """
    youtube_tool = YouTubeSearchTool()
    raw_result = youtube_tool.run(query)

    # raw_result is usually a stringified list; make this robust:
    video_ids = ast.literal_eval(raw_result)

    links = list(video_ids)[:2]

    chunks = []
    for i, link in enumerate(links, start=1):
        loader = YoutubeLoader.from_youtube_url(
            link,
            add_video_info=False,
        )
        docs = loader.load()
        if not docs:
            continue

        text = docs[0].page_content

        # Hard truncate per video (e.g., first 1500 characters)
        text = text[:1500]

        chunks.append(f"Video {i} ({link}) snippet:\n{text}")

    if not chunks:
        return "No transcripts found for that query."

    # Final size cap on the whole tool output
    result = "\n\n".join(chunks)
    return result[:4000]

tools.append(fetch_youtube_content)

Google search api key

In [ ]:
from langchain_community.utilities import SearchApiAPIWrapper
from langchain_core.tools import Tool

SEARCHAPI_API_KEY = "my_api_key"

# 1) Google Web Search
google_web = SearchApiAPIWrapper(
    searchapi_api_key=SEARCHAPI_API_KEY,
    engine="google",            # standard Google web search
)

google_web_tool = Tool(
    name="google_web_search",
    description="General Google web search via SearchApi.io.",
    func=google_web.run,
)

# 2) Google News
google_news = SearchApiAPIWrapper(
    searchapi_api_key=SEARCHAPI_API_KEY,
    engine="google_news",       # Google News vertical
)

google_news_tool = Tool(
    name="google_news_search",
    description="Latest news headlines and articles via Google News (SearchApi.io).",
    func=google_news.run,
)

# 3) Google Scholar
google_scholar = SearchApiAPIWrapper(
    searchapi_api_key=SEARCHAPI_API_KEY,
    engine="google_scholar",    # Google Scholar vertical
)

google_scholar_tool = Tool(
    name="google_scholar_search",
    description="Academic papers and scholarly articles via Google Scholar (SearchApi.io).",
    func=google_scholar.run,
)

# Adding to my tools list
tools.append(google_web_tool)
tools.append(google_news_tool)
tools.append(google_scholar_tool)

# Prompt using lang chain hub

In [163]:
from langchain_classic import hub

prompt = hub.pull("hwchase17/react")

# Agent = LLM + Tool + Prompt

In [ ]:
from langchain_classic.agents import initialize_agent

agent = initialize_agent(
    tools=tools,
    llm=llm,
    prompt=prompt,
    verbose=True
)

print(agent.run("Top news in AI using duck duck go, wikipedia"))



> Entering new AgentExecutor chain...
To find the top news in AI, I should start by searching for the latest news and developments in the field of Artificial Intelligence. 

Action: duckduckgo_search
Action Input: Top news in AI
Observation: Explore the latest artificial intelligence news with Reuters - from AI breakthroughs and technology trends to regulation, ethics, business and global impact. From Gemini 3.0 to the beginnings of AI browsers, here are the top AI news of 2025. Get the latest AI news , updates, and breakthroughs. Stay informed with real-time coverage on artificial intelligence trends, research, and industry developments. Artificial Intelligence News . Everything on AI including futuristic robots with artificial intelligence, computer models of human intelligence and more. Get the week's most important AI news fast. Our curated roundup covers LLMs, robotics, cutting-edge tools, and real-world breakthroughs worldwide.
Thought:Since I've found some general information 